# Launch Sites Locations Analysis with Folium

### Objectives
This lab contains the following tasks:

* **TASK 1:** Mark all launch sites on a map
* **TASK 2:** Mark the success/failed launches for each site on the map
* **TASK 3:** Calculate the distances between a launch site to its proximities


Let's first import required Python packages for this lab:

In [3]:
!pip3 install folium
!pip3 install wget

In [4]:
import folium
import wget
import pandas as pd
import math

In [5]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Task 1: Mark all launch sites on a map

First, let's try to add each site's location on a map using site's latitude and longitude coordinates

The following dataset with the name spacex_launch_geo.csv is an augmented dataset with latitude and longitude added for each site.

In [6]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

100% [................................................................................] 7710 / 7710

Now, you can take a look at what are the coordinates for each site.

In [7]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.

We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,

In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Now, let's add a circle for each launch site in data frame launch_sites



Create and add folium.Circle and folium.Marker for each launch site on the site map


In [10]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

#Add Circle markers and regular markers for each launch site
for index, row in spacex_df.iterrows():
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,  # Radius in meters
        color='blue',
        fill=True,
        fill_color='blue',
        popup=row['Launch Site']  # Add the launch site name as a popup label
    ).add_to(site_map)
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],  # Add the launch site name as a popup label
        icon=folium.Icon(icon='rocket', color='red')  # Custom icon (optional)
    ).add_to(site_map)

#Display the map (in a Jupyter notebook or similar environment)
site_map

Regular Markers: Represent each launch site with a marker that you can click to see the site name.

## Task 2: Mark the success/failed launches for each site on the map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not

In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.

Let's first create a MarkerCluster object

In [12]:
marker_cluster = MarkerCluster()

In [13]:
# Step 1: Create the 'marker_color' column based on the 'Class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Step 2: Verify the new column
print(spacex_df[['class', 'marker_color']].head(25))

    class marker_color
0       0          red
1       0          red
2       0          red
3       0          red
4       0          red
5       0          red
6       0          red
7       0          red
8       0          red
9       0          red
10      0          red
11      0          red
12      0          red
13      0          red
14      0          red
15      0          red
16      0          red
17      1        green
18      1        green
19      0          red
20      1        green
21      1        green
22      1        green
23      0          red
24      1        green


In [14]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster

In [15]:
from folium.plugins import MarkerCluster

#Create the marker_color column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

#Initialize the map centered at the average coordinates of all launch sites
nasa_coordinate = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

#Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

#Add each launch site as a folium.Marker to the marker_cluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],  # Add the launch site name as a popup label
        icon=folium.Icon(color=row['marker_color'])  # Use the marker color from the DataFrame
    ).add_to(marker_cluster)

#Display the map (in a Jupyter notebook or similar environment)
site_map


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

## TASK 3: Calculate the distances between a launch site to its proximities

Next, we need to explore and analyze the proximities of launch sites.

Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

You can calculate the distance between two points on the map based on their Lat and Long values using the following method:

In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [22]:
from folium import DivIcon
from IPython.display import display

# Create the marker_color column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Initialize the map centered at the average coordinates of all launch sites
nasa_coordinate = [spacex_df['Lat'].mean(), spacex_df['Long'].mean()]
print(f"Map center coordinates: {nasa_coordinate}")
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add each launch site as a folium.Marker to the marker_cluster
for index, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=row['Launch Site'],
        icon=folium.Icon(color=row['marker_color'])
    ).add_to(marker_cluster)

# Add MousePosition to the map to track cursor position
folium.plugins.MousePosition().add_to(site_map)

# Manually mark a point on the closest coastline
coastline_point = [28.3922, -80.6077]  # Latitude and Longitude of a coastline point

# Add the coastline point to the map
folium.Marker(
    location=coastline_point,
    popup="Closest Coastline",
    icon=folium.Icon(color='blue', icon='info-sign')
).add_to(site_map)

# Calculate the distance between the coastline point and the closest launch site
min_distance = float('inf')
closest_launch_site = None
closest_launch_site_coord = None

for index, row in spacex_df.iterrows():
    launch_site_location = (row['Lat'], row['Long'])
    distance = calculate_distance(launch_site_location[0], launch_site_location[1], coastline_point[0], coastline_point[1])
    if distance < min_distance:
        min_distance = distance
        closest_launch_site = row['Launch Site']
        closest_launch_site_coord = launch_site_location

"""# Add a marker for the coastline point with the distance shown using DivIcon
distance_marker = folium.Marker(
    location=coastline_point,
    icon=DivIcon(
        icon_size=(200, 36),  # Adjust the size as needed
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{min_distance:.2f} KM</b></div>',
    )
)
distance_marker.add_to(site_map)"""

import folium

# assuming you have the coordinate and distance variables

distance_marker = folium.Marker(
    location=coastline_point,
    icon=folium.DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance),
    )
)

# add the marker to the map
distance_marker.add_to(site_map)

print(f"Closest Launch Site: {closest_launch_site}")
print(f"Distance to Closest Coastline Point: {min_distance:.2f} km")

import folium

# Define the coastline coordinates.
coastline_coordinates = [(40.7128, -74.0060), (40.7128, -74.0060), (40.7128, -74.0060)]

# Define the launch site coordinates.
launch_site_coordinates = (40.7128, -74.0060)

# Create a folium map centered on the launch site coordinates.
site_map = folium.Map(location=launch_site_coordinates, zoom_start=12)

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate.
coordinates = [closest_launch_site_coord, coastline_point]
lines = folium.PolyLine(locations=coordinates, weight=2, color='blue')


# Add the coastline to the map.
site_map.add_child(lines)


# Save the map to an HTML file
html_file = "spacex_launch_map_with_distance_marker.html"
site_map.save(html_file)
print(f"Map saved to {html_file}")

# Display the map in Jupyter Notebook
display(site_map)


Map center coordinates: [29.648980044464285, -87.74225182964287]
Closest Launch Site: CCAFS LC-40
Distance to Closest Coastline Point: 19.15 km
Map saved to spacex_launch_map_with_distance_marker.html


Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc.

In [42]:
# Define the coordinates for each launch site
launch_sites = {
    'CCAFS LC-40': (28.5622, -80.5774),
    'VAFB SLC-4E': (34.6327, -120.6103),
    'KSC LC-39A': (28.6084, -80.6042),
    # Add more launch sites as needed
}

# Define the coordinates for railways, highways, coastlines, and cities
infrastructure = {
    'Railways': [
        (28.5733, -80.6480),  # Railway near CCAFS LC-40
        (34.6452, -120.6231),  # Railway near VAFB SLC-4E
        # Add more railway coordinates as needed
    ],
    'Highways': [
        (28.5623, -80.5774),  # Highway near CCAFS LC-40
        (34.6272, -120.6063),  # Highway near VAFB SLC-4E
        # Add more highway coordinates as needed
    ],
    'Coastlines': [
        (28.3922, -80.6077),  # Coastline near CCAFS LC-40
        (34.4333, -120.4833),  # Coastline near VAFB SLC-4E
        # Add more coastline coordinates as needed
    ],
    'Cities': [
        (28.5383, -81.3792),  # Orlando (near CCAFS LC-40)
        (34.2133, -119.2000),  # Ventura (near VAFB SLC-4E)
        # Add more city coordinates as needed
    ]
}

# Create a Folium map
m = folium.Map(location=[30, -100], zoom_start=4)

# Draw lines between launch sites and their closest infrastructure
for launch_site, (lat, lon) in launch_sites.items():
    for infrastructure_type, coords in infrastructure.items():
        distances = [(calculate_distance(lat, lon, coord[0], coord[1]), coord) for coord in coords]
        if distances:
            nearest_distance, nearest_coord = min(distances, key=lambda x: x[0])
            # Draw a line between the launch site and its closest infrastructure
            folium.PolyLine([[lat, lon], nearest_coord], color='blue').add_to(m)


# Add markers for each launch site
for launch_site, (lat, lon) in launch_sites.items():
    launch_marker = folium.Marker(location=[lat, lon], popup=launch_site)
    launch_marker.add_to(m)

    # Calculate the nearest infrastructure for each launch site
    for infrastructure_type, coords in infrastructure.items():
        distances = [(calculate_distance(lat, lon, coord[0], coord[1]), coord) for coord in coords]
        nearest_distance, nearest_coord = min(distances, key=lambda x: x[0])

        # Create a marker for the nearest infrastructure
        infrastructure_marker = folium.Marker(location=list(nearest_coord), popup=f"Nearest {infrastructure_type}: {nearest_distance:.2f} km")
        infrastructure_marker.add_to(m)

m